#Setup

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import figure
import seaborn as sns
import pandas as pd
import operator
from math import log
from pandas import Series, DataFrame
from pylab import rcParams
import pyreadr

import sklearn
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold # import KFold
from sklearn import datasets

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

from matplotlib.lines import Line2D
import re
import pickle

/Users/Abood/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Using Google Drive to store files

In [2]:
passing_grades = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D",
                  "D-", "S", "CR"]
passing_pattern = r"A[+-]*|B[+-]*|C[+-]*|D[+-]*|S|CR"
success_grades = ["A+", "A", "A-", "B+"]

# Exploration with toy dataset "cs_ug.csv"

In [3]:
# raw_data = pd.read_csv('../data/last_five_years.csv', header=0)
# result = pyreadr.read_r('../data/merged_data.rds')
# raw_data = result[None]
raw_data = pd.read_feather('../data/merged_data.fthr')

In [4]:
raw_data.loc[[10]]

,acad_career,strm,class_nbr,stdnt_enrl_status,unt_taken,unt_prgrss,grading_basis_dt,unt_billing,grading_scheme_enr,crse_grade_off,...,crse_id,effdt,major1,major2,major3,major4,major5,major6,major7,sex
10,UG,1184.0,21562.0,E,2.0,2.0,2018-01-10,2.0,REG,A+,...,202740.0,None,None,None,None,None,None,None,None,M


In [5]:
raw_data.columns

Index(['acad_career', 'strm', 'class_nbr', 'stdnt_enrl_status', 'unt_taken',
       'unt_prgrss', 'grading_basis_dt', 'unt_billing', 'grading_scheme_enr',
       'crse_grade_off', 'crse_grade_input', 'earn_credit', 'oprid',
       'last_upd_dt_stmp', 'last_upd_tm_stmp', 'emplid', 'subject',
       'ssr_component', 'catalog_nbr', 'crse_acad_group', 'crse_acad_org',
       'crse_id', 'effdt', 'major1', 'major2', 'major3', 'major4', 'major5',
       'major6', 'major7', 'sex'],
      dtype='object')

Winter is 4, Spring is 6, Fall is 2, Summer is 8?
e.g. 1198 is 18-19, summer quarter. 1196 is 18-19, spring quarter

In [ ]:
#all allowed quarters
terms = raw_data.strm.unique()
terms = list(filter(lambda x: x >= 1162, terms)) 
print(terms)

In [6]:
def course_name (row):
    if row['subject'] != None and row['catalog_nbr'] != None:
        return row['subject']+row['catalog_nbr']
    return ""
def student_gender (row):
    if row['sex'] == "F":
        return 1
    return 0
def dropped_function (row):
    if row['stdnt_enrl_status'] == 'D':
        return 1
    return 0

In [7]:
new_courses = raw_data.loc[raw_data['acad_career'] == "UG"]
# new_courses['course_name'] = new_courses.apply(lambda row: course_name(row), axis=1)
# new_courses['female'] = new_courses.apply(lambda row: student_gender(row), axis=1)
# new_courses['dropped'] = new_courses.apply(lambda row: dropped_function(row), axis=1)
# new_courses = new_courses.drop(['ssr_component', 'crse_grade_off', 'last_upd_dt_stmp',
#                                 'last_upd_tm_stmp', 'class_nbr', 'stdnt_enrl_status',
#                                 'unt_taken', 'unt_prgrss', 'grading_basis_dt', 'unt_billing',
#                                 'grading_scheme_enr', 'crse_acad_group', 'crse_acad_org', 'crse_id',
#                                 'effdt', 'major1', 'major2', 'major3', 'major4', 'major5', 'major6', 'major7',
#                                 'catalog_nbr', 'oprid', 'sex'], axis=1)
# new_courses

In [9]:
new_courses_mod = new_courses.drop(['ssr_component', 'crse_acad_group', 'crse_grade_off', 'last_upd_dt_stmp',
                                'last_upd_tm_stmp', 'class_nbr',
                                'grading_basis_dt', 'effdt', 'unt_prgrss',
                                'grading_scheme_enr', 'crse_id',
                                'major1', 'major2', 'major3', 'major4', 'major5', 'major6', 'major7', 'oprid'], axis=1)

In [12]:
new_courses_mod.to_csv('../data/clean_full.csv')

In [13]:
new_courses_mod

,acad_career,strm,stdnt_enrl_status,unt_taken,unt_billing,crse_grade_input,earn_credit,emplid,subject,catalog_nbr,crse_acad_org,sex
0,UG,1182.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,MATH,51,MATH,M
1,UG,1182.0,E,0.0,0.0,None,N,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CHEM,31X,CHEMISTRY,M
2,UG,1182.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CHEM,31X,CHEMISTRY,M
3,UG,1182.0,E,4.0,4.0,A-,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,LINGUIST,35,LINGUISTIC,M
4,UG,1182.0,E,0.0,0.0,None,N,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,MATH,51,MATH,M
...,...,...,...,...,...,...,...,...,...,...,...,...
2667926,UG,1204.0,E,2.0,2.0,None,Y,ZZY9cXv8E+kma1UpxorU3h6lmrEAShLb3z/uaqmJdU+YHG...,AFRICAST,195,AFRICAST,F
2667927,UG,1204.0,E,1.0,1.0,None,Y,ZZY9cXv8E+kma1UpxorU3h6lmrEAShLb3z/uaqmJdU+YHG...,UAR,42B,VPUE,F
2667928,UG,1204.0,E,1.0,1.0,None,Y,ZZY9cXv8E+kma1UpxorU3h6lmrEAShLb3z/uaqmJdU+YHG...,UAR,56,VPUE,F
2667929,UG,1204.0,E,3.0,3.0,None,Y,ZZY9cXv8E+kma1UpxorU3h6lmrEAShLb3z/uaqmJdU+YHG...,HUMRTS,108,ICA,F


In [ ]:
# new_courses is the entries post 2010
new_courses = raw_data.loc[raw_data['strm'].isin(terms)]
unique_course_list = new_courses.course_name.unique()
num_students = len(new_courses.emplid.unique())
num_courses = len(unique_course_list)

In [ ]:
def getStudentFeatures(course):
    course_entries = cs_ugrad.loc[cs_ugrad['course_name'] == course] # all course entries that are about the certain course we want
    print(course_entries['crse_grade_input'].unique())
    letter_entries = course_entries[(course_entries['crse_grade_input'] == 'A+') | (course_entries['crse_grade_input'] == 'A') | (course_entries['crse_grade_input'] == 'A-') | 
                                 (course_entries['crse_grade_input'] == 'B+') | (course_entries['crse_grade_input'] == 'B') | (course_entries['crse_grade_input'] == 'B-') |
                                 (course_entries['crse_grade_input'] == 'C+') | (course_entries['crse_grade_input'] == 'C') | (course_entries['crse_grade_input'] == 'C-') |
                                 (course_entries['crse_grade_input'] == 'D+') | (course_entries['crse_grade_input'] == 'D') | (course_entries['crse_grade_input'] == 'D-') |
                                 (course_entries['crse_grade_input'] == 'NP') | (course_entries['crse_grade_input'] == 'W')]
    a = np.zeros(shape=(len(letter_entries),num_courses)) # all 0 initialized features matrix
    X = pd.DataFrame(a,columns=unique_course_list) # same as above, but in pandas to allow indexing by course name
    y = np.zeros(shape=(len(letter_entries),1)) # outcome vector
    for counter, idx in enumerate(letter_entries.index): # loop over all class entried
    # fill outcome vector y
      grade = letter_entries.loc[[idx]]['crse_grade_input']
      # print(grade.unique())
      gradeStr = grade.tolist()[0]
      if (gradeStr == "A+" or gradeStr == "A" or
          gradeStr == "A-" or gradeStr == "B+"):
        y[counter] = 1
        # fill predictor matrix X
        std_id = letter_entries.loc[[idx]]['emplid'] # Get the student id
        quarter = letter_entries.loc[[idx]]['strm'] # Get the term
        student_courses = cs_ugrad.loc[cs_ugrad['emplid'] == std_id.tolist()[0]] # Get all courses the student took
        student_courses = student_courses.loc[student_courses['strm'] < quarter.tolist()[0]] # Filter the courses to be only before the course of interest
        student_courses = student_courses.loc[student_courses['strm'] >= 1100] # Filter the courses to only be after 2010
        for j in student_courses.index:# Loop over the student's courses
            course_name = student_courses.loc[[j]]['course_name'] # Get the course name
            grade = student_courses.loc[[j]]['crse_grade_input']
            gradeStr = grade.tolist()[0]
            if (gradeStr == "A+" or gradeStr == "A" or
                gradeStr == "A-" or gradeStr == "B+" or gradeStr == "B" or
                gradeStr == "B-" or gradeStr == "C+" or gradeStr == "C" or
                gradeStr == "C-" or gradeStr == "D+" or gradeStr == "D" or
                gradeStr == "D-"):
              X.iloc[counter, X.columns.get_loc(course_name.tolist()[0])] = 1 # Set the course name in X to the new value (or if it was already passed keep the value)
            dev_break = True
    return X,y.ravel()

In [ ]:
X_110 = pd.read_pickle(DATA_PATH+"/CS110_X.pkl")
y = np.load(DATA_PATH+"/CS110_y.npy")
X_229 = pd.read_pickle(DATA_PATH+"/CS229_X.pkl")
y_229 = np.load(DATA_PATH+"/CS229_y.npy")
X_230 = pd.read_pickle(DATA_PATH+"/CS230_X.pkl")
y_230 = np.load(DATA_PATH+"/CS230_y.npy")

In [ ]:
X_110,y_110 = getStudentFeatures('CS110')
X_229,y_229 = getStudentFeatures('CS229')

In [ ]:
X_230,y_230 = getStudentFeatures('CS230')

In [ ]:
collections.Counter(y_229)

# Try sklearn logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Use the X predictor matrix and y outcome vector from 
# getStudentFeatures, fit a logistic regression model and return
# the list of coefficients, default sorted by absolute value

# Params: sort = ["pos", "neg", "abs"] to sort by largest positive, negative, or
# absolute value of coefficients
def get_coefs(X, y, sort = "abs"):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = LogisticRegression(solver="lbfgs")
  model.fit(X_train, y_train)
  train_score = model.score(X_test,y_test) 
  print("training accuracy: %s" % train_score)
  coefs = [(X.columns[i], model.coef_[0][i]) for i in range(len(X.columns))]
  if sort:
    if sort == "pos":
      coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
    if sort == "neg":
      coefs = sorted(coefs, key = lambda x: x[1], reverse = False)
    if sort == "abs":
      coefs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
  return coefs

In [ ]:
CS110coefs = get_coefs(X_110, y_110)
CS110coefs[:20]

In [ ]:
CS230coefs = get_coefs(X_230, y_230)
CS230coefs[:20]

In [ ]:
import collections
CS229coefs = get_coefs(X_229, y_229)
CS229coefs[:20]

More Logistic regression

In [ ]:
CS110coefs = get_coefs(X_high, y, 'pos')
CS110coefs[:20]

In [ ]:
X, y = getStudentFeatures("CS106B")
CS106Bcoefs = get_coefs(X, y)
CS106Bcoefs[:20]

GLENN: It seems like at least for CS110 and CS106B, the largest predictor is having taken that course beforehand, and it's a negative coefficient. Seems strange. Let's look into that.

In [ ]:
course_entries = cs_ugrad.loc[cs_ugrad['course_name'] == "CS110"]
counts = course_entries['emplid'].value_counts()
mult_emplids = counts[counts > 1].index
mult_students = course_entries.loc[course_entries['emplid'].isin(mult_emplids)]
mult_students.sort_values(['emplid','strm'])

In [ ]:
# GLENN:
# Not sure what's going on with emplid 22166: [NC, drop, W, drop]
# shouldn't be showing up as containing a passing grade but the
# other results look right

for emplid in mult_emplids:
  one_student = course_entries.loc[course_entries['emplid'] == emplid].sort_values(['strm'])
  if one_student['crse_grade_input'].str.contains(passing_pattern, na = False).any():
    print(one_student[['strm', 'crse_grade_input', 'dropped', 'emplid']])

GLENN: Looks like this may be due to a small number of students taking and passing the course with a low grade, and then considering taking it again for a better grade but dropping it instead.


GLENN: Let's look at positive coefficients for now. I picked a few courses as samples below: the prereqs kind of make sense but we definitely need to find some ways to refine this further. Maybe doing something with the number of students who took a certain prereq? If only a small number of students took a prereq that could lead to huge coefficients?

In [ ]:
X, y = getStudentFeatures("CS110")
CS110coefs = get_coefs(X, y, "pos")
CS110coefs[:20]

# contains CS107 and CS107E in top 20

In [ ]:
X, y = getStudentFeatures("CS229")
CS229coefs = get_coefs(X, y, "pos")
CS229coefs[:20]

# nothing stands out to me here; maybe it's because CS229 is more
# of a math course so the strong predictors might be MATH51 or
# other non-CS courses?

In [ ]:
X, y = getStudentFeatures("CS221")
CS221coefs = get_coefs(X, y, "pos")
CS221coefs[:20]

# contains CS224S, which is a "postreq" (221 is prereq for 224S)

In [ ]:
X, y = getStudentFeatures("CS547")
CS224Scoefs = get_coefs(X, y, "pos")
CS224Scoefs[:20]

# contains CS224S, which is a "postreq" (221 is prereq for 224S)

In [ ]:
X, y = getStudentFeatures("CS224N")
CS224Ncoefs = get_coefs(X, y, "pos")
CS224Ncoefs[:20]

# contains CS221, 229, 229A

In [ ]:
X = pd.read_pickle(DATA_PATH+"/CS224N_X.pkl")
y = np.load(DATA_PATH+"/CS224N_y.npy")
CS224Ncoefs = get_coefs(X, y, "pos")
CS224Ncoefs[:20]

In [ ]:
X, y = getStudentFeatures("CS230")
CS230coefs = get_coefs(X, y, "pos")
CS230coefs[:20]

# doesn't contain CS229, which I would have expected

GLENN: How should we interpret negative coefficients? The model suggests that passing these other courses beforehand predicts that you won't pass the course in question, but that seems weird. There isn't an obvious reason why passing CS224U should make you less prepared for CS110, for example.

YITIAN: I thought negative weights should just be interpreted as less likely to be helpful in passing certain class. Although implemented a bit differently in scikit, it still follows the general sigmoid function where the negative weights would make the sigmoid really small (since all the inputs X is positive) therefore less likely to be helpful. 

In [ ]:
X, y = getStudentFeatures("CS110")
CS110coefs = get_coefs(X, y, "neg")
CS110coefs[:20]

Things to try:
- PCA
- Collaborative filtering to detect feature vectors of classes, then cluster them? (Is this just more complicated PCA?)
- Adjust $\lambda$ for normalization
- Change it so the coefficients are all $e^{\text{coef}}$ (all positive), requires some kind of transformation to log space?
- Full dataset instead of just CS, find predictors in other departments
- Weighted regression or some kind of significance indicator to account for the predictors with very few data points

Interpretations:
- What do we do with negative coefficients?

# **Compare different Models**

In [ ]:
def runDifferentModels(X,y):
  # split X and y into training and testing sets
  X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, y, test_size=0.3)
  X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
  print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  #"Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process","Decision Tree", "Random Forest", "Neural Net", "AdaBoost","Naive Bayes", "QDA"
  models = []
  models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
  models.append(('LDA', LinearDiscriminantAnalysis()))
  models.append(('GP', GaussianProcessClassifier()))
  models.append(('RF', RandomForestClassifier()))
  models.append(('NN', MLPClassifier()))
  models.append(('AB', AdaBoostClassifier()))
  models.append(('QDA', QuadraticDiscriminantAnalysis()))
  models.append(('CART', DecisionTreeClassifier()))
  models.append(('NB', GaussianNB()))
  models.append(('K_M', KNeighborsClassifier(metric = 'minkowski')))
  models.append(('K_C', KNeighborsClassifier(metric = 'chebyshev')))
  models.append(('K_E', KNeighborsClassifier(metric = 'euclidean')))
  models.append(('K_Man', KNeighborsClassifier(metric = 'manhattan')))
  models.append(('K_Mat', KNeighborsClassifier(metric = 'matching')))
  models.append(('K_J', KNeighborsClassifier(metric = 'jaccard')))
  models.append(('K_D', KNeighborsClassifier(metric = 'dice')))
  models.append(('K_K', KNeighborsClassifier(metric = 'kulsinski')))
  models.append(('S_Lin', SVC(kernel='linear')))
  models.append(('S_RBF', SVC(kernel='rbf')))
  models.append(('S_Sig', SVC(kernel="sigmoid")))
  models.append(('S_Pol', SVC(kernel="poly")))
  

  # Evaluate each model through 10-fold cross-validation
  results = []
  names = []
  number = 1
  for name, model in models:
      kfold = KFold(n_splits=10, random_state=42)
      cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
      results.append(cv_results)
      names.append(name)
      msg = "%i) %s: Mean = %f with std = (%f)" % (number, name, cv_results.mean(), cv_results.std())
      number += 1
      print(msg)
  # Compare Algorithms
  matplotlib.rcParams.update({'font.size': 40})
  fig = plt.figure()
  fig = plt.figure(figsize=(28,20))
  plt.title('Algorithm Comparison')
  ax = fig
  medianprops = {'color': 'magenta', 'linewidth': 6}
  boxprops = {'color': 'black', 'linewidth': 4, 'linestyle': '-'}
  whiskerprops = {'color': 'black', 'linewidth': 4, 'linestyle': '-'}
  capprops = {'color': 'black', 'linewidth': 4, 'linestyle': '-'}
  flierprops = {'color': 'black', 'marker': 'x'}
  plt.boxplot(results,
           medianprops=medianprops,
           boxprops=boxprops,
           whiskerprops=whiskerprops,
           capprops=capprops,
           flierprops=flierprops)
#   ax.set_xticklabels(names)
  plt.xlabel('Model Number', fontsize=40)
  plt.ylabel('Accuracy', fontsize=40)
  plt.savefig(DATA_PATH+'/visualizations/comparison_110_B+.png')

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X_110)

# X_train (10 input features, 70% of full dataset), X_val (10 input features, 15% of full dataset), X_test (10 input features, 15% of full dataset)
# Y_train (1 label, 70% of full dataset), Y_val (1 label, 15% of full dataset), Y_test (1 label, 15% of full dataset)
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, y_110, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [ ]:
runDifferentModels(X_110,y_110)

In [ ]:
plt.savefig(DATA_PATH+'/visualizations/comparison_229.png')

# **NeurralNet**

In [ ]:
def runNeuralNet(X,y):
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scale = min_max_scaler.fit_transform(X)

    # X_train (10 input features, 70% of full dataset), X_val (10 input features, 15% of full dataset), X_test (10 input features, 15% of full dataset)
    # Y_train (1 label, 70% of full dataset), Y_val (1 label, 15% of full dataset), Y_test (1 label, 15% of full dataset)
    X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, y, test_size=0.3)
    X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
    print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

    # values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
    # for param in values:
    #     # model = Sequential([Dense(32, activation='relu', input_shape=(58,)), Dense(32, activation='relu'), Dense(1, activation='sigmoid'),]) # described sequentially, layer-by-layer with 32, 32, and 1 neurons; 58 input features
    #     print("Param = " + str(param))
        
    # SGD Optimizer
    param = 1e-3
    model = Sequential([Dense(100, activation='relu', input_shape=(1022,), kernel_regularizer=l1(param)), Dense(100, activation='relu', kernel_regularizer=l1(param)), Dense(100, activation='relu', kernel_regularizer=l1(param)), Dense(1, activation='sigmoid'),]) # described sequentially, layer-by-layer with 32, 32, and 1 neurons; 58 input features
    # model = Sequential([Dense(32, activation='relu', input_shape=(58,), kernel_regularizer=l2(param)), Dense(32, activation='relu', kernel_regularizer=l2(param)), Dense(1, activation='sigmoid'),]) # described sequentially, layer-by-layer with 32, 32, and 1 neurons; 58 input features
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy']) 
    history = model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_data=(X_val, Y_val))

    # Plot epoch vs. accuracy plot
    fig = plt.figure(figsize=(28,20))
    plt.plot(history.history['acc'], linewidth=7)
    plt.plot(history.history['val_acc'], linewidth=7)
    plt.plot(history.history['loss'], linewidth=7)
    plt.plot(history.history['val_loss'], linewidth=7)
    plt.rcParams.update({'font.size': 40})
    plt.title('SGD Model Training Loss and Accuracy')
    plt.ylabel('Accuracy/Loss', fontsize=40)
    plt.xlabel('Epoch', fontsize=40)
    plt.legend(['Acc_Train', 'Acc_Val', 'Loss_Train', 'Loss_Val'], loc='upper right')
    plt.savefig(DATA_PATH+'/visualizations/SGD_110_B+.png')
    print("SGD Accuracy: " + str(model.evaluate(X_test, Y_test)[1]))

    param = 1e-2
    # Adam optimizer
    model_2 = Sequential([Dense(100, activation='relu', input_shape=(1022,), kernel_regularizer=l2(param)), Dense(100, activation='relu'), Dense(100, activation='relu'), Dense(1, activation='sigmoid'),])
    model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history2 = model_2.fit(X_train, Y_train, batch_size=32, epochs=100, validation_data=(X_val, Y_val))

    # Plot epoch vs. accuracy plot
    plt.figure()
    fig = plt.figure(figsize=(28,20))
    plt.plot(history2.history['acc'], linewidth=7)
    plt.plot(history2.history['val_acc'], linewidth=7)
    plt.plot(history2.history['loss'], linewidth=7)
    plt.plot(history2.history['val_loss'], linewidth=7)
    plt.rcParams.update({'font.size': 20})
    # plt.figure(figsize=(14,10))
    plt.title('Adam Model Training Loss and Accuracy')
    plt.ylabel('Accuracy/Loss', fontsize=40)
    plt.xlabel('Epoch', fontsize=40)
    plt.legend(['Acc_Train', 'Acc_Val', 'Loss_Train', 'Loss_Val'], loc='upper right')
    plt.savefig(DATA_PATH+'/visualizations/Adam_110_B+.png')
    print("Adam Accuracy: " + str(model_2.evaluate(X_test, Y_test)[1]))

    print("Success!")

runNeuralNet(X_110, y_110)

# **PCA**

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 

pca = PCA(n_components=50)
principalComponents = pca.fit_transform(X_110)
principalDf = pd.DataFrame(data = principalComponents)

CS110coefs = get_coefs(principalDf, y_110)
CS110coefs[:20]
pca = PCA().fit(X_110)
#Plotting the Cumulative Summation of the Explained Variance
matplotlib.rcParams.update({'font.size': 40})
fig = plt.figure()
fig = plt.figure(figsize=(28,20))
plt.plot(np.cumsum(pca.explained_variance_ratio_), lw=7)
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Dataset Explained Variance')
plt.savefig(DATA_PATH+"/visualizations/PCA_110_B+.png")

# **Baseline Analysis**

In [ ]:
import pickle
DATA_PATH = "/content/drive/My Drive/CS129 Carta project/data"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
# Use the X predictor matrix and y outcome vector from 
# getStudentFeatures, fit a logistic regression model and return
# the list of coefficients, default sorted by absolute value

# Params: sort = ["pos", "neg", "abs"] to sort by largest positive, negative, or
# absolute value of coefficients

# modified get_coefs function to include training and test sets as well as output model scores
def get_coefs(X_train, X_test, y_train, y_test, sort = "abs"):
  model = LogisticRegression(solver="lbfgs", C=1, max_iter=100, multi_class="multinomial") # multinomial enables softmax 
  model.fit(X_train, y_train)
  train_score = model.score(X_train,y_train)
  test_score = model.score(X_test,y_test)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)

  coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
  if sort:
    if sort == "pos":
      coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
    if sort == "neg":
      coefs = sorted(coefs, key = lambda x: x[1], reverse = False)
    if sort == "abs":
      coefs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
  return coefs

In [ ]:
#split the data to train and test set
X = pd.read_pickle(DATA_PATH+"/CS142_X.pkl")
y = np.load(DATA_PATH+"/CS142_y.npy")
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
coefs = get_coefs(X_train, X_test, y_train, y_test, "pos")
coefs[:20]

In [ ]:
#plot learning curves
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    matplotlib.rcParams.update({'font.size': 40})
    fig = plt.figure()
    fig = plt.figure(figsize=(28,20))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score", lw=7)
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score", lw=7)

    plt.legend(loc="best")
    return plt

In [ ]:
collections.Counter(y_230)

In [ ]:
title = "Learning Cruves (CS230)"
X = pd.read_pickle(DATA_PATH+"/CS229_X.pkl")
y = np.load(DATA_PATH+"/CS229_y.npy")


# Cross validation with 10 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=2, test_size=0.1, random_state=42)

estimator = LogisticRegression(solver="lbfgs", multi_class="multinomial")
plot_learning_curve(estimator, title, X_230, y_230, cv=cv, n_jobs=8)
plt.savefig(DATA_PATH+"/visualizations/learning_curve_230_B+.png")

In [ ]:
def plot_validation_curves(title, param_name, param_range, train_scores, test_scores):
  train_scores_mean = np.mean(train_scores, axis=1)
  train_scores_std = np.std(train_scores, axis=1)
  test_scores_mean = np.mean(test_scores, axis=1)
  test_scores_std = np.std(test_scores, axis=1)
  
  matplotlib.rcParams.update({'font.size': 40})
  fig = plt.figure()
  fig = plt.figure(figsize=(28,20))
  plt.title(title)
  plt.xlabel(param_name)
  plt.ylabel("Score")
  lw = 7
  
  plt.plot(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
  plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
  plt.plot(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
  plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
  plt.legend(loc="best")
  return plt

In [ ]:
#plot validation curves
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve

# X = pd.read_pickle(DATA_PATH+"/CS106B_X_plus.pkl")
# y = np.load(DATA_PATH+"/CS106B_y_plus.npy")

param_range= [50, 100, 200, 400]
estimator = LogisticRegression(solver="lbfgs", multi_class="multinomial")
train_scores, test_scores = validation_curve(estimator, X_230, y_230, param_name="max_iter", param_range=param_range, cv=5, scoring="accuracy", n_jobs=8)

title="Validation Curve of CS230 with Logistic Regression"
plot_validation_curves(title,"max_iter",param_range,train_scores,test_scores)
plt.savefig(DATA_PATH+"/visualizations/validation_curve_230_B+.png")

**CS110**

In [ ]:
title = "Learning Cruves (CS110)"
X = pd.read_pickle(DATA_PATH+"/CS110_X.pkl")
y = np.load(DATA_PATH+"/CS110_y.npy")

# Cross validation with 10 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = LogisticRegression(solver="lbfgs", multi_class="multinomial")
plot_learning_curve(estimator, title, X, y, cv=cv, n_jobs=4)
plt.show()

In [ ]:
#checking iterations
param_range= [50, 100, 200, 400]
estimator = LogisticRegression(solver="lbfgs", multi_class="multinomial")
train_scores, test_scores = validation_curve(estimator, X, y, param_name="max_iter", param_range=param_range, cv=5, scoring="accuracy", n_jobs=1)

title="Iteration Curve of CS110 with Logistic Regression"
plot_validation_curves(title,"max_iter",param_range,train_scores,test_scores)
plt.show()

In [ ]:
#checking regularizations
param_range= [1.0, 2.0, 4.0, 8.0]
estimator = LogisticRegression(solver="lbfgs", multi_class="multinomial")
train_scores, test_scores = validation_curve(estimator, X, y, param_name="C", param_range=param_range, cv=5, scoring="accuracy", n_jobs=1)

title="Regularization Curve of CS110 with Logistic Regression"
plot_validation_curves(title,"C",param_range,train_scores,test_scores)
plt.show()

In [ ]:
#get weights
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
coefs = get_coefs(X_train, X_test, y_train, y_test, "pos")
coefs[:20]

# **Data Loading and Saving**

In [ ]:
#load data for CS106B and save them to google drive for future use
X, y = getStudentFeatures("CS106B")
X.to_pickle(DATA_PATH+"/CS106B_X.pkl")
np.save(DATA_PATH+"/CS106B_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS110")
X.to_pickle(DATA_PATH+"/CS110_X.pkl")
np.save(DATA_PATH+"/CS110_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS140")
X.to_pickle(DATA_PATH+"/CS140_X.pkl")
np.save(DATA_PATH+"/CS140_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS155")
X.to_pickle(DATA_PATH+"/CS155_X.pkl")
np.save(DATA_PATH+"/CS155_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS161")
X.to_pickle(DATA_PATH+"/CS161_X.pkl")
np.save(DATA_PATH+"/CS161_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS142")
X.to_pickle(DATA_PATH+"/CS142_X.pkl")
np.save(DATA_PATH+"/CS142_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS224S")
X.to_pickle(DATA_PATH+"/CS224S_X.pkl")
np.save(DATA_PATH+"/CS224S_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS229")
X.to_pickle(DATA_PATH+"/CS229_X.pkl")
np.save(DATA_PATH+"/CS229_y.npy",y)

In [ ]:
X, y = getStudentFeatures("CS129")
X.to_pickle(DATA_PATH+"/CS129_X.pkl")
np.save(DATA_PATH+"/CS129_y.npy",y)

In [ ]:
from sklearn.metrics import accuracy_score
model = LogisticRegression(solver="lbfgs")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)


# **Lasso Model**

In [ ]:
#This is a bad model. Leave here for reference
def get_coefs_lasso(X_train, X_test, y_train, y_test):
  model = Lasso(alpha=0.0001,precompute=True,max_iter=1000,positive=True, random_state=9999, selection='random')
  model.fit(X_train, y_train)
  coefs = [(X_train.columns[i], model.coef_[i]) for i in range(len(X_train.columns))]
  coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
  train_score = model.score(X_train,y_train)
  test_score = model.score(X_test,y_test)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)

  return coefs

In [ ]:
X = pd.read_pickle(DATA_PATH+"/CS142_X.pkl")
y = np.load(DATA_PATH+"/CS142_y.npy")
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
coefs = get_coefs_lasso(X_train, X_test, y_train, y_test)
coefs[:20]

# **SVM**

In [ ]:
from sklearn.svm import SVC
def get_coefs_svm(X_train, X_test, y_train, y_test):
  model = SVC(kernel='rbf',gamma='scale')
  model.fit(X_train, y_train)
  train_score = model.score(X_train,y_train)
  test_score = model.score(X_test,y_test)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)


In [ ]:
X = pd.read_pickle(DATA_PATH+"/CS142_X.pkl")
y = np.load(DATA_PATH+"/CS142_y.npy")
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
coefs = get_coefs_svm(X_train, X_test, y_train, y_test)

# **Larger Dataset**

In [ ]:
X, y = getStudentFeatures("CS161")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
coefs = get_coefs(X_train, X_test, y_train, y_test, "pos")
coefs[:20]

In [ ]:
coefs = get_coefs(X, y, "pos")
coefs[:20]

In [ ]:
X.to_pickle(DATA_PATH+"/CS161_X_plus.pkl")
np.save(DATA_PATH+"/CS229_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS106B")
X.to_pickle(DATA_PATH+"/CS106B_X_plus.pkl")
np.save(DATA_PATH+"/CS106B_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS110")
X.to_pickle(DATA_PATH+"/CS110_X_plus.pkl")
np.save(DATA_PATH+"/CS110_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS140")
X.to_pickle(DATA_PATH+"/CS140_X_plus.pkl")
np.save(DATA_PATH+"/CS140_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS155")
X.to_pickle(DATA_PATH+"/CS155_X_plus.pkl")
np.save(DATA_PATH+"/CS155_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS142")
X.to_pickle(DATA_PATH+"/CS142_X_plus.pkl")
np.save(DATA_PATH+"/CS142_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS224S")
X.to_pickle(DATA_PATH+"/CS224S_X_plus.pkl")
np.save(DATA_PATH+"/CS224S_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS229")
X.to_pickle(DATA_PATH+"/CS229_X_plus.pkl")
np.save(DATA_PATH+"/CS229_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS129")
X.to_pickle(DATA_PATH+"/CS129_X_plus.pkl")
np.save(DATA_PATH+"/CS129_y_plus.npy",y)

In [ ]:
X, y = getStudentFeatures("CS161")
X.to_pickle(DATA_PATH+"/CS161_X_plus.pkl")
np.save(DATA_PATH+"/CS161_y_plus.npy",y)

# GLENN: Reducing number of coefficients (Lasso, PCA, and other attempts)

In [ ]:
# need a different library to get p-values
import statsmodels.api as sm

cs_ugrad_plus = pd.read_csv('/content/drive/My Drive/CS129 Carta project/data/cs_ug_plus.csv', header=0)

In [ ]:
X, y = getStudentFeatures2("CS110", cs_ugrad_plus)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# logit_model = sm.Logit(y, X)
# result = logit_model.fit()
# print(result.summary())

Doesn't work (singular matrix). Let's try LASSO logistic regression.

In [ ]:
from pprint import pprint

model = LogisticRegression(penalty="l1", solver="saga", C=1, max_iter=300) 
model.fit(X_train, y_train)
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)
coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
coefs_nonzero = sum([x[1] != 0 for x in coefs])
coefs_zero = sum(x[1] == 0 for x in coefs)
print("training accuracy: %s" % train_score)
print("test accuracy: %s" % test_score)
print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))

print("Largest absolute coefficients:")
pprint(coefs_abs[:10])
print("Largest positive coefficients:")
pprint(coefs_pos[:10])
print("Largest negative coefficients:")
pprint(coefs_neg[:10])


In [ ]:
model = LogisticRegression(penalty="l1", solver="saga", C=0.1, max_iter=300) 
model.fit(X_train, y_train)
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)
coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
coefs_nonzero = sum([x[1] != 0 for x in coefs])
coefs_zero = sum(x[1] == 0 for x in coefs)
print("training accuracy: %s" % train_score)
print("test accuracy: %s" % test_score)
print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))

print("Largest absolute coefficients:")
pprint(coefs_abs[:10])
print("Largest positive coefficients:")
pprint(coefs_pos[:10])
print("Largest negative coefficients:")
pprint(coefs_neg[:10])

In [ ]:
for i in [10, 3, 1, 0.3, 0.1, 0.03, 0.01]:
  model = LogisticRegression(penalty="l1", solver="saga", C=i, max_iter=300) 
  model.fit(X_train, y_train)
  train_score = model.score(X_train, y_train)
  test_score = model.score(X_test, y_test)
  coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
  coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
  coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
  coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
  coefs_nonzero = sum([x[1] != 0 for x in coefs])
  coefs_zero = sum(x[1] == 0 for x in coefs)
  print("***** C = %f *****" % i)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)
  print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))
  print("Largest absolute coefficients:")
  pprint(coefs_abs[:3])
  print("Largest positive coefficients:")
  pprint(coefs_pos[:3])
  print("Largest negative coefficients:")
  pprint(coefs_neg[:3])


In [ ]:
X, y = getStudentFeatures2("CS107", cs_ugrad_plus)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

for i in [10, 3, 1, 0.3, 0.1, 0.03, 0.01]:
  model = LogisticRegression(penalty="l1", solver="saga", C=i, max_iter=300) 
  model.fit(X_train, y_train)
  train_score = model.score(X_train, y_train)
  test_score = model.score(X_test, y_test)
  coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
  coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
  coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
  coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
  coefs_nonzero = sum([x[1] != 0 for x in coefs])
  coefs_zero = sum(x[1] == 0 for x in coefs)
  print("***** C = %f *****" % i)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)
  print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))
  print("Largest absolute coefficients:")
  pprint(coefs_abs[:3])
  print("Largest positive coefficients:")
  pprint(coefs_pos[:3])
  print("Largest negative coefficients:")
  pprint(coefs_neg[:3])

In [ ]:
# accuracy seems weird, check ground truth

print(sum([score == 1 for score in y_train]) / len(y_train))

This model isn't doing much more than just predicting based on the intercept.

# Whole dataset

In [ ]:
cs_ugrad = pd.read_csv('/content/drive/My Drive/CS129 Carta project/data/cs_ug_plus.csv', header=0)
top50courses = cs_ugrad.groupby('course_name').count().nlargest(50, 'strm').index.values
topCourses = np.delete(top50courses,[8,14,18,29,30,38,39,40,45])
topCourses = np.append(topCourses,'CS230')

In [ ]:
#for topCourses
results = []

for course in topCourses:
  X,y = getStudentFeatures(course)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = LogisticRegression(solver="lbfgs")
  model.fit(X_train, y_train)
  #study coefficients
  coefs = [(X.columns[i], model.coef_[0][i]) for i in range(len(X.columns))]
  coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
  coefs_top20 = coefs[:20]
  prereqInCoef = PrereqPredicted(course, coefs_top20)

  #model evaluation
  train_score = model.score(X_train, y_train)
  y_pred_train = model.predict(X_train)
  train_precision, train_recall, train_fscore, train_support = precision_recall_fscore_support(y_train, y_pred_train, average="binary")
  test_score = model.score(X_test, y_test)
  y_pred_test = model.predict(X_test)
  test_precision, test_recall, test_fscore, test_support = precision_recall_fscore_support(y_test, y_pred_test, average="binary")
  
  #save results
  result = {
    'course_name': course, 
    'train_accuracy': train_score, 
    'train_precision': train_precision, 
    'train_recall': train_recall, 
    'train_fscore': train_fscore, 
    'test_accuracy': test_score, 
    'test_precision': test_precision, 
    'test_recall': test_recall, 
    'test_fscore': test_fscore,
    'official_prereq_in_coefs': prereqInCoef,
    'coefs': coefs_top20
  }
  print(result)
  results.append(result)


results = pd.DataFrame(results)
results.to_pickle(DATA_PATH+"/topResults.pkl")

In [ ]:
results.to_csv(DATA_PATH+"/topResults.csv")

In [ ]:
results.to_pickle(DATA_PATH+"/topResultsWith230.pkl")

In [ ]:
cs_ugrad = pd.read_csv('/content/drive/My Drive/CS129 Carta project/data/cs_ug_plus.csv', header=0)
top50courses = cs_ugrad.groupby('course_name').count().nlargest(50, 'strm').index.values
topCourses = np.delete(top50courses,[8,14,18,29,30,38,39,40,45])
topCourses = np.append(topCourses,'CS230')

#encode the data for scatter plot
def encode_course(course):
    encode = np.where(topCourses == course)
    if not encode[0]:
      encode = -1
    else:
      encode = encode[0][0]
    return encode

In [ ]:
#generate coordinates for one course
def generate_coordinates(course, prereq):
  x = [encode_course(course)] * len(prereq)
  y = [encode_course(i) for i in prereq]
  return x, y


In [ ]:
#generate scatter plot

results = pd.read_pickle(DATA_PATH+"/topResultsWith230.pkl")
prereq_db = pickle.load(open(DATA_PATH+"/prereq_db.pkl", "rb"))
f = plt.figure()

for row_index, row in results.iterrows():
  course = row['course_name']
  #draw official prereqs
  dp_code = re.findall("[A-Z]+", course)[0]
  official_prereq = [dp_code + str(i) for i in prereq_db[course]]
  x_0, y_0 = generate_coordinates(course, official_prereq)
  plt.scatter(x_0, y_0, c='r')

  #draw prereq in coefs
  prereq_in_coefs = row['official_prereq_in_coefs']
  x_1, y_1 = generate_coordinates(course, prereq_in_coefs)
  plt.scatter(x_1, y_1, c='y')

  #draw coefs
  coefs = row['coefs']
  coefs_courses = [i[0] for i in coefs]
  coefs_weights = [i[1] for i in coefs]
  x_2, y_2 = generate_coordinates(course, coefs_courses)
  plt.scatter(x_2, y_2, c='g', s=coefs_weights)

ax = plt.subplot(1,1,1)
ax.set_ylim(bottom=0.)
ax.set_xlim(left=0.)
plt.xlabel("top 40 Courses")
plt.ylabel("prereqs")
ax.set_xticks([24,41])
ax.set_xticklabels(['CS229','CS230'])
plt.yticks([])
legend_elements = [Line2D([0], [0], marker='o', color='w', label='official_prereq',
                          markerfacecolor='r', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='predict_coefs',
                          markerfacecolor='g', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='official_prereq_in_coefs',
                          markerfacecolor='y', markersize=15)
                   ]
ax.legend(handles=legend_elements)
plt.show()

f.savefig(DATA_PATH+"/result.pdf", bbox_inches='tight')






# ExploreCourses API

In [ ]:
! pip install explorecourses

In [ ]:
from explorecourses import *
from explorecourses import filters
connect = CourseConnection()
CScourses = connect.get_courses_by_department("CS")
EEcourses = connect.get_courses_by_department("EE")
STATScourses = connect.get_courses_by_department("STATS")
CMEcourses = connect.get_courses_by_department("CME")
MATHcourses = connect.get_courses_by_department("MATH")

In [ ]:
#scraping numbers from course description and append the course code to it as a dictionary.
# prerequisites_db = {
#     'course code': []
# }

for course in courses:
#  print(course.code)
  prerequisites = [float(s) for s in re.findall(r'-?\d+\.?\d*', course.description)]
  prerequisites = [round(x) for x in prerequisites]
#  print(prerequisites)
  prerequisites_db['EE'+course.code] = prerequisites

In [ ]:
#data saving
f = open(DATA_PATH+"/prereq_db.pkl", "wb")
pickle.dump(prerequisites_db, f)
f.close()

In [ ]:
# a helper function to check if coefs contains any official prereqs
prereq_db = pickle.load(open(DATA_PATH+"/prereq_db.pkl", "rb"))
def PrereqPredicted(course, coefs):
  prereqInCoef = []
  for i in coefs:
    course_code = re.findall("\d+", i[0])[0]
    for j in prereq_db[course]:
      if j == int(course_code):
        prereqInCoef.append(i[0])
#    prereqInCoef = [i[0] for j in prereq_db[course] if int(course_code) == j]
  return prereqInCoef

In [ ]:
#manually check the official prereq dict
prereq_db = pickle.load(open(DATA_PATH+"/prereq_db.pkl", "rb"))


In [ ]:
for course in topCourses:
  print(course + ": " + str(prereq_db[course]))

In [ ]:
prereq_db['CS105']=[]